In [2]:
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Activation, Concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import backend, models
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model, Sequential

from IPython.display import Image, display
import random
from fastai.vision import *
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt 
import random
from fastai.vision import *


from tensorflow.keras.applications import VGG16, MobileNet
from keras.applications.vgg16 import preprocess_input
import warnings
warnings.filterwarnings("ignore")

print("Library loaded")

In [3]:
train_dir = "../input/cat-and-dog/training_set/training_set"
test_dir = "../input/cat-and-dog/test_set/test_set/"

In [4]:
categories = os.listdir(train_dir)
print(str(len(categories)), "Categories: ", categories)
category_count = len(categories)

In [5]:
image = load_img("../input/cat-and-dog/training_set/training_set/dogs/dog.10.jpg")
plt.imshow(image)
plt.axis("off")
plt.show()

imagedata = img_to_array(image)
shape = imagedata.shape
print("Figure are, ", shape)

In [6]:
datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_data = datagen.flow_from_directory(train_dir, target_size=(224,224))
test_data = datagen.flow_from_directory(test_dir, target_size=(224,224))

In [7]:
train_data.class_indices

In [8]:
Train_groups = len(train_data)
Test_groups = len(test_data)

In [9]:
imgs, labels = next(train_data)

In [26]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(30, 30))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

In [11]:
plotImages(imgs)
print(labels)

In [12]:
model = Sequential()

model.add(Conv2D(64, (3,3), padding='same', input_shape=(224,224, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.35))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
Augment_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

Augmentation_train = Augment_datagen.flow_from_directory(train_dir, target_size=(224,224))

In [15]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [16]:
history Sequentialdel.fit_generator(
            Augmentation_train,
            epochs=20,
            validation_data = test_data, verbose=2, callbacks=[es])

In [17]:
# Accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [18]:
model.evaluate(test_data, verbose=1)

In [19]:
test_imgs, test_labels = next(test_data)

In [20]:
plotImages(test_imgs)
print(test_labels)

In [21]:
predictions = model.predict(x=test_data, verbose=0)

In [22]:
np.round(predictions)

In [23]:
cm = confusion_matrix(y_true=test_data.classes, y_pred=np.argmax(predictions, axis=1))

In [24]:
cm

In [25]:
pred = model.predict(test_data)
pred = np.argmax(pred,axis=1)

In [26]:
labels = (train_data.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

print(f"The first 5 predictions: {pred[:5]}")

## VGG16

In [9]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_dir, target_size=(224,224), classes=['cats', 'dogs'], batch_size=10)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_dir, target_size=(224,224), classes=['cats', 'dogs'], batch_size=10, shuffle=False)

In [10]:
vgg16_model = tf.keras.applications.vgg16.VGG16()

In [11]:
vgg16_model.summary()

In [13]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [16]:
for layer in model.layers:
    layer.trainable = False

In [17]:
model.add(Dense(units=2, activation='softmax'))

In [18]:
model.summary()

In [19]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [21]:
model.fit(x=train_batches, 
         steps_per_epoch = len(train_batches),
         epochs=50,
         verbose=2, callbacks=[es])

In [23]:
model.evaluate(test_data, verbose=1)

In [24]:
test_imgs, test_labels = next(test_data)

In [27]:
plotImages(test_imgs)
print(test_labels)